In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.tsa.api as sm
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt

from statsmodels.tsa.statespace.sarimax import SARIMAX

In [2]:
df=pd.read_csv('Global Electricity Statistics.csv')

In [3]:
df.head()

,Country,Features,Region,1980,1981,1982,1983,1984,1985,1986,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Algeria,net generation,Africa,6.683,7.65,8.824,9.615,10.537,11.569,12.214,...,53.9845,56.3134,60.39972,64.68244,66.75504,71.49546,72.10903,76.685,72.73591277,77.53072719
1,Angola,net generation,Africa,0.905,0.906,0.995,1.028,1.028,1.028,1.088,...,6.03408,7.97606,9.21666,9.30914,10.203511,10.67604,12.83194,15.4,16.6,16.429392
2,Benin,net generation,Africa,0.005,0.005,0.005,0.005,0.005,0.005,0.005,...,0.04612,0.08848,0.22666,0.31056,0.26004,0.3115,0.19028,0.2017,0.22608,0.24109728
3,Botswana,net generation,Africa,0.443,0.502,0.489,0.434,0.445,0.456,0.538,...,0.33,0.86868,2.17628,2.79104,2.52984,2.8438,2.97076,3.0469,2.05144,2.18234816
4,Burkina Faso,net generation,Africa,0.098,0.108,0.115,0.117,0.113,0.115,0.122,...,0.86834,0.98268,1.11808,1.43986,1.5509,1.64602,1.6464,1.72552,1.647133174,1.761209666


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1610 entries, 0 to 1609
Data columns (total 45 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Country   1610 non-null   object
 1   Features  1610 non-null   object
 2   Region    1610 non-null   object
 3   1980      1582 non-null   object
 4   1981      1582 non-null   object
 5   1982      1582 non-null   object
 6   1983      1582 non-null   object
 7   1984      1582 non-null   object
 8   1985      1582 non-null   object
 9   1986      1582 non-null   object
 10  1987      1582 non-null   object
 11  1988      1582 non-null   object
 12  1989      1582 non-null   object
 13  1990      1582 non-null   object
 14  1991      1582 non-null   object
 15  1992      1582 non-null   object
 16  1993      1582 non-null   object
 17  1994      1582 non-null   object
 18  1995      1582 non-null   object
 19  1996      1582 non-null   object
 20  1997      1582 non-null   object
 21  1998      1582

In [6]:
df.isnull().sum()/len(df)*100

Country     0.000000
Features    0.000000
Region      0.000000
1980        1.739130
1981        1.739130
1982        1.739130
1983        1.739130
1984        1.739130
1985        1.739130
1986        1.739130
1987        1.739130
1988        1.739130
1989        1.739130
1990        1.739130
1991        1.739130
1992        1.739130
1993        1.739130
1994        1.739130
1995        1.739130
1996        1.739130
1997        1.739130
1998        1.739130
1999        1.739130
2000        1.739130
2001        1.739130
2002        1.739130
2003        1.739130
2004        1.739130
2005        1.739130
2006        1.739130
2007        1.739130
2008        1.739130
2009        1.739130
2010        1.739130
2011        1.739130
2012        1.739130
2013        1.739130
2014        1.739130
2015        1.739130
2016        1.739130
2017        1.739130
2018        1.739130
2019        1.739130
2020        1.925466
2021        1.614907
dtype: float64

In [7]:
# Since there is just a small percentage of null values in each year, null values are dropped. 
df.dropna(axis=0, inplace=True)

In [9]:
df.isnull().values.any()

False

In [11]:
# Explore categorical variables.
df['Country'].unique()

array(['        Algeria', '        Angola', '        Benin',
       '        Botswana', '        Burkina Faso', '        Burundi',
       '        Cabo Verde', '        Cameroon',
       '        Central African Republic', '        Chad',
       '        Comoros', '        Congo-Brazzaville',
       '        Congo-Kinshasa', "        Cote d'Ivoire",
       '        Djibouti', '        Egypt', '        Equatorial Guinea',
       '        Eritrea', '        Eswatini', '        Ethiopia',
       '        Gabon', '        Gambia, The', '        Ghana',
       '        Guinea', '        Guinea-Bissau', '        Kenya',
       '        Lesotho', '        Liberia', '        Libya',
       '        Madagascar', '        Malawi', '        Mali',
       '        Mauritania', '        Mauritius', '        Morocco',
       '        Mozambique', '        Namibia', '        Niger',
       '        Nigeria', '        Reunion', '        Rwanda',
       '        Saint Helena', '        Sao Tome and Pri

In [14]:
df['Region'].value_counts()

Region
Africa                     398
Asia & Oceania             322
Europe                     315
Central & South America    312
Middle East                 98
Eurasia                     91
North America               42
Name: count, dtype: int64

In [15]:
df['Features'].value_counts()

Features
net generation          226
net consumption         226
imports                 226
exports                 226
installed capacity      226
distribution losses     226
net imports             222
Name: count, dtype: int64

In [18]:
# Un-pivot years into a column and create the 'Energy value' column. 
df2=pd.melt(df, id_vars=['Country', 'Features', 'Region'], var_name='Year', value_name='Energy value')
df2.head()

,Country,Features,Region,Year,Energy value
0,Algeria,net generation,Africa,1980,6.683
1,Angola,net generation,Africa,1980,0.905
2,Benin,net generation,Africa,1980,0.005
3,Botswana,net generation,Africa,1980,0.443
4,Burkina Faso,net generation,Africa,1980,0.098


In [20]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66276 entries, 0 to 66275
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Country       66276 non-null  object
 1   Features      66276 non-null  object
 2   Region        66276 non-null  object
 3   Year          66276 non-null  object
 4   Energy value  66276 non-null  object
dtypes: object(5)
memory usage: 2.5+ MB


In [27]:
# Year and Energy value are object data type, so they are turned into integer and float. 
# the column 'Energy value' had non numerical values that are transfomed into NANs with the error method 'coerce'.
df2.Year = df2.Year.astype('int64') 
df2['Energy value'] = pd.to_numeric(df2['Energy value'], errors='coerce').astype(float)

In [28]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66276 entries, 0 to 66275
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country       66276 non-null  object 
 1   Features      66276 non-null  object 
 2   Region        66276 non-null  object 
 3   Year          66276 non-null  int64  
 4   Energy value  61514 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 2.5+ MB


In [39]:
df2.isnull().sum()/len(df2)*100

Country         0.000000
Features        0.000000
Region          0.000000
Year            0.000000
Energy value    7.185105
dtype: float64

In [34]:
# Dataframe with the null values is created to understand if there is a pattern in the null values, such as in certain years, features category or country. 
null_df = df2[df2.isnull().any(axis=1)]
null_df

,Country,Features,Region,Year,Energy value
17,Eritrea,net generation,Africa,1980,NaN
36,Namibia,net generation,Africa,1980,NaN
48,South Sudan,net generation,Africa,1980,NaN
57,Armenia,net generation,Eurasia,1980,NaN
58,Azerbaijan,net generation,Eurasia,1980,NaN
...,...,...,...,...,...
66138,"Germany, West",distribution losses,Europe,2021,NaN
66178,Hawaiian Trade Zone,distribution losses,Asia & Oceania,2021,NaN
66250,French Guiana,distribution losses,Central & South America,2021,NaN
66252,Guadeloupe,distribution losses,Central & South America,2021,NaN


In [38]:
null_df['Country'].value_counts()

Country
        Hawaiian Trade Zone             245
        South Sudan                     224
        Germany, West                   217
        Germany, East                   217
        Former Yugoslavia               210
        Former U.S.S.R.                 210
        Former Czechoslovakia           203
        Kosovo                          196
        Former Serbia and Montenegro    196
        Serbia                          182
        Montenegro                      182
        Timor-Leste                     161
        Palestinian Territories         119
        Eritrea                          98
        Slovakia                         91
        Czechia                          91
        Uzbekistan                       84
        Armenia                          84
        Azerbaijan                       84
        Belarus                          84
        Georgia                          84
        Kazakhstan                       84
        Kyrgyzstan      

In [36]:
null_df['Features'].value_counts()

Features
net generation          701
installed capacity      701
distribution losses     701
exports                 669
imports                 668
net consumption         661
net imports             661
Name: count, dtype: int64

In [37]:
null_df['Year'].unique()

array([1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021], dtype=int64)

In [40]:
# Null values are dropped. 
df2.dropna(axis=0, inplace=True)

In [41]:
df2.isnull().values.any()

False